In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

In [3]:
#train data
X_train = train_df.drop(['label'], axis=1).values.reshape(-1, 28, 28)   
y_train = train_df['label'].values

In [4]:
#test data
X_test = test_df.values.reshape(-1, 28, 28)

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

In [6]:
#convert to tensor
X_train = torch.Tensor(X_train).view(-1, 1, 28, 28)
y_train = torch.Tensor(y_train).long()
X_test = torch.Tensor(X_test).view(-1, 1, 28, 28)

In [14]:
#create dataset
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, X_test)

In [15]:
#create dataloader
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [27]:
#load pretrained ResNet50 model
from torchvision import models
model = models.resnet50(pretrained=True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=True)
model.fc = nn.Linear(2048, 10)
model.to(device)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=Fal

In [28]:
#define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [29]:
#train the model
epochs = 20
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        #forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        #backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 2 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                  .format(epoch+1, epochs, i+1, len(train_loader), loss.item()))

Epoch [1/20], Step [2/657], Loss: 2.5020
Epoch [1/20], Step [4/657], Loss: 2.0302
Epoch [1/20], Step [6/657], Loss: 1.7937
Epoch [1/20], Step [8/657], Loss: 1.5096
Epoch [1/20], Step [10/657], Loss: 1.9819
Epoch [1/20], Step [12/657], Loss: 1.2996
Epoch [1/20], Step [14/657], Loss: 1.3832
Epoch [1/20], Step [16/657], Loss: 1.1562
Epoch [1/20], Step [18/657], Loss: 0.9291
Epoch [1/20], Step [20/657], Loss: 0.8753
Epoch [1/20], Step [22/657], Loss: 0.7277
Epoch [1/20], Step [24/657], Loss: 0.6149
Epoch [1/20], Step [26/657], Loss: 0.8367
Epoch [1/20], Step [28/657], Loss: 0.7838
Epoch [1/20], Step [30/657], Loss: 0.9121
Epoch [1/20], Step [32/657], Loss: 0.3368
Epoch [1/20], Step [34/657], Loss: 0.3999
Epoch [1/20], Step [36/657], Loss: 0.5234
Epoch [1/20], Step [38/657], Loss: 0.6398
Epoch [1/20], Step [40/657], Loss: 0.3017
Epoch [1/20], Step [42/657], Loss: 0.8542
Epoch [1/20], Step [44/657], Loss: 0.4110
Epoch [1/20], Step [46/657], Loss: 0.3604
Epoch [1/20], Step [48/657], Loss: 0.4

In [33]:
#test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    result = []
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        #append the predicted labels to result
        result.extend(predicted.tolist())
        
    

In [34]:
#convert result to dataframe
result = pd.DataFrame(result)
result.columns = ['Label']
#add the image id to the dataframe
result['ImageId'] = range(1, len(result)+1)
#swap the columns
result = result[['ImageId', 'Label']]

In [35]:
result.to_csv('submission.csv', index=False)